In [ ]:
# Phase 2: Real-Time Drift Detection
# ====================================

import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
from pathlib import Path
import time

# Project imports
from src.calibration import DriftCollector
from src.probes import ProbeSuite, QubitSelector
from src.qec import RepetitionCode, QECExperimentRunner
from src.analysis import DriftErrorAnalyzer
from src.utils import QPUBudgetTracker, load_experiment_results

# IBM Quantum imports
from qiskit_ibm_runtime import QiskitRuntimeService

print("Phase 2 imports loaded successfully")

## 2.1 Load Baseline Results & Initialize Backend

In [ ]:
# Load Phase 1 baseline results for comparison
baseline_results = load_experiment_results('../data/experiments/phase1_baseline_results.json')
df_baseline = pd.read_parquet('../data/experiments/phase1_baseline.parquet')

print(f"Loaded {len(df_baseline)} baseline experiments")
print(f"Baseline backend: {baseline_results['experiment_metadata']['backend']}")

In [ ]:
# Initialize service and backend (use same backend as baseline)
service = QiskitRuntimeService(channel="ibm_quantum")
baseline_backend_name = baseline_results['experiment_metadata']['backend']

try:
    backend = service.backend(baseline_backend_name)
    print(f"Using baseline backend: {backend.name}")
except:
    backend = service.least_busy(simulator=False, operational=True, min_num_qubits=27)
    print(f"Baseline backend unavailable, using: {backend.name}")

# Initialize budget tracker
budget_tracker = QPUBudgetTracker(total_budget_seconds=600)
print(f"Budget remaining: {budget_tracker.remaining_budget():.1f} seconds")

## 2.2 Initialize Probe Suite

The `ProbeSuite` implements lightweight 30-shot diagnostics:
- **T1 Probe**: Single delay time to estimate T1
- **Readout Probe**: |0⟩ and |1⟩ preparation + measurement
- **RB Probe**: 2-3 Clifford sequences for gate fidelity estimate

In [ ]:
# Initialize probe suite with 30 shots per probe
probe_suite = ProbeSuite(
    backend=backend,
    shots_per_probe=30,
    probes=['t1', 'readout', 'rb']
)

print(f"Probe suite initialized with {len(probe_suite.probes)} probes")
print(f"Estimated time per full probe: {probe_suite.estimate_time():.2f} seconds")

## 2.3 Collect Initial Calibration & Probe Data

In [ ]:
# Get current calibration snapshot
drift_collector = DriftCollector(backend)
initial_calibration = drift_collector.collect_calibration_snapshot()

print(f"Initial calibration collected at: {initial_calibration['timestamp']}")

# Identify candidate qubits for probing (top 20 from calibration)
selector_static = QubitSelector(backend, strategy='static')
candidate_qubits = selector_static.select_qubits(
    n_qubits=20,
    calibration_data=initial_calibration
)['qubits']

print(f"Candidate qubits for probing: {candidate_qubits}")

In [ ]:
# Run initial probes on candidate qubits
print("Running initial probe suite...")
initial_probe_results = probe_suite.run_probes(
    qubits=candidate_qubits,
    budget_tracker=budget_tracker
)

print(f"\nProbe results collected for {len(initial_probe_results['qubit_data'])} qubits")
print(f"Budget used so far: {budget_tracker.used_budget():.1f} seconds")

# Display probe results summary
probe_df = pd.DataFrame(initial_probe_results['qubit_data'])
print("\nProbe Results Summary:")
print(probe_df[['qubit', 't1_probe', 'readout_error_probe', 'rb_fidelity_probe']].head(10))

## 2.4 Compare Static vs Real-Time Qubit Selection

In [ ]:
# Static selection (from daily calibration)
selector_static = QubitSelector(backend, strategy='static')
static_selection_d5 = selector_static.select_qubits(
    n_qubits=9,  # For distance-5 code
    calibration_data=initial_calibration
)

# Real-time selection (from probe data)
selector_rt = QubitSelector(backend, strategy='realtime')
rt_selection_d5 = selector_rt.select_qubits(
    n_qubits=9,
    probe_data=initial_probe_results
)

print("Distance-5 Qubit Selection Comparison:")
print(f"  Static (calibration): {static_selection_d5['qubits']}")
print(f"  Real-time (probes):   {rt_selection_d5['qubits']}")
print(f"  Overlap: {len(set(static_selection_d5['qubits']) & set(rt_selection_d5['qubits']))} qubits")

## 2.5 Drift-Tracking Experiment Loop

Run a series of experiments with periodic probe refreshes to track drift:
1. Probe → Select qubits → Run QEC → Wait
2. Repeat N times over experimental session

In [ ]:
# Drift tracking parameters
N_ITERATIONS = 5  # Number of probe-QEC cycles
WAIT_BETWEEN_ITERATIONS = 600  # 10 minutes between iterations
CODE_DISTANCE = 5
N_SYNDROME_ROUNDS = 3
SHOTS_PER_CIRCUIT = 1000

# Storage for drift tracking
drift_tracking_data = []
rt_qec_results = []
static_qec_results = []

print(f"Starting drift tracking experiment:")
print(f"  Iterations: {N_ITERATIONS}")
print(f"  Wait between: {WAIT_BETWEEN_ITERATIONS}s")
print(f"  Code distance: {CODE_DISTANCE}")

In [ ]:
# Main drift tracking loop
runner = QECExperimentRunner(backend=backend, budget_tracker=budget_tracker)

for iteration in range(N_ITERATIONS):
    print(f"\n{'='*50}")
    print(f"Iteration {iteration + 1}/{N_ITERATIONS}")
    print(f"Time: {datetime.now(timezone.utc).isoformat()}")
    print(f"Budget remaining: {budget_tracker.remaining_budget():.1f}s")
    
    # Step 1: Run probes
    print("Running probes...")
    probe_results = probe_suite.run_probes(
        qubits=candidate_qubits,
        budget_tracker=budget_tracker
    )
    
    # Store drift data
    drift_tracking_data.append({
        'iteration': iteration,
        'timestamp': probe_results['timestamp'],
        'probe_data': probe_results['qubit_data']
    })
    
    # Step 2: Select qubits using RT strategy
    rt_qubits = selector_rt.select_qubits(
        n_qubits=2 * CODE_DISTANCE - 1,
        probe_data=probe_results
    )['qubits']
    
    # Step 3: Run QEC with RT-selected qubits
    print(f"Running QEC with RT qubits: {rt_qubits}")
    rep_code_rt = RepetitionCode(distance=CODE_DISTANCE, qubits=rt_qubits)
    circuit_rt = rep_code_rt.build_circuit(
        n_syndrome_rounds=N_SYNDROME_ROUNDS,
        initial_state='0',
        measure_final=True
    )
    
    job_id_rt = runner.submit_batch(
        circuits=[circuit_rt],
        metadata=[{'strategy': 'rt', 'iteration': iteration}],
        shots=SHOTS_PER_CIRCUIT
    )
    
    # Step 4: Run QEC with static-selected qubits (for comparison)
    static_qubits = static_selection_d5['qubits']
    print(f"Running QEC with static qubits: {static_qubits}")
    rep_code_static = RepetitionCode(distance=CODE_DISTANCE, qubits=static_qubits)
    circuit_static = rep_code_static.build_circuit(
        n_syndrome_rounds=N_SYNDROME_ROUNDS,
        initial_state='0',
        measure_final=True
    )
    
    job_id_static = runner.submit_batch(
        circuits=[circuit_static],
        metadata=[{'strategy': 'static', 'iteration': iteration}],
        shots=SHOTS_PER_CIRCUIT
    )
    
    # Collect results
    results_rt = runner.collect_results(job_id_rt)
    results_static = runner.collect_results(job_id_static)
    
    rt_qec_results.append({
        'iteration': iteration,
        'qubits': rt_qubits,
        'counts': results_rt['counts'][0],
        'timestamp': datetime.now(timezone.utc).isoformat()
    })
    
    static_qec_results.append({
        'iteration': iteration,
        'qubits': static_qubits,
        'counts': results_static['counts'][0],
        'timestamp': datetime.now(timezone.utc).isoformat()
    })
    
    # Wait before next iteration (except for last)
    if iteration < N_ITERATIONS - 1:
        print(f"Waiting {WAIT_BETWEEN_ITERATIONS}s before next iteration...")
        time.sleep(WAIT_BETWEEN_ITERATIONS)

print(f"\n{'='*50}")
print("Drift tracking experiment complete!")

## 2.6 Analyze Drift Magnitude

In [ ]:
# Extract drift time series for each qubit
drift_analysis = []

for qubit in candidate_qubits:
    qubit_series = []
    for entry in drift_tracking_data:
        qubit_data = next((q for q in entry['probe_data'] if q['qubit'] == qubit), None)
        if qubit_data:
            qubit_series.append({
                'iteration': entry['iteration'],
                'timestamp': entry['timestamp'],
                't1': qubit_data.get('t1_probe'),
                'readout_error': qubit_data.get('readout_error_probe'),
                'rb_fidelity': qubit_data.get('rb_fidelity_probe')
            })
    
    if len(qubit_series) >= 2:
        t1_values = [s['t1'] for s in qubit_series if s['t1']]
        re_values = [s['readout_error'] for s in qubit_series if s['readout_error']]
        
        drift_analysis.append({
            'qubit': qubit,
            't1_mean': np.mean(t1_values) if t1_values else None,
            't1_std': np.std(t1_values) if t1_values else None,
            't1_drift_pct': (np.std(t1_values) / np.mean(t1_values) * 100) if t1_values else None,
            'readout_mean': np.mean(re_values) if re_values else None,
            'readout_std': np.std(re_values) if re_values else None
        })

df_drift = pd.DataFrame(drift_analysis)
print("Drift Analysis Summary:")
print(df_drift.sort_values('t1_drift_pct', ascending=False).head(10))

## 2.7 Compare RT vs Static QEC Performance

In [ ]:
from src.qec import SyndromeDecoder

# Decode results and compute logical error rates
decoder = SyndromeDecoder(distance=CODE_DISTANCE)

rt_error_rates = []
static_error_rates = []

for rt_res, static_res in zip(rt_qec_results, static_qec_results):
    # RT analysis
    rt_analysis = decoder.analyze_results(
        counts=rt_res['counts'],
        initial_state='0',
        n_rounds=N_SYNDROME_ROUNDS
    )
    rt_error_rates.append({
        'iteration': rt_res['iteration'],
        'strategy': 'RT',
        'logical_error_rate': rt_analysis['logical_error_rate'],
        'syndrome_error_rate': rt_analysis['syndrome_error_rate']
    })
    
    # Static analysis
    static_analysis = decoder.analyze_results(
        counts=static_res['counts'],
        initial_state='0',
        n_rounds=N_SYNDROME_ROUNDS
    )
    static_error_rates.append({
        'iteration': static_res['iteration'],
        'strategy': 'Static',
        'logical_error_rate': static_analysis['logical_error_rate'],
        'syndrome_error_rate': static_analysis['syndrome_error_rate']
    })

df_comparison = pd.DataFrame(rt_error_rates + static_error_rates)
print("RT vs Static Comparison:")
print(df_comparison.groupby('strategy')['logical_error_rate'].agg(['mean', 'std', 'min', 'max']))

In [ ]:
# Statistical significance test
from scipy import stats

rt_rates = [r['logical_error_rate'] for r in rt_error_rates]
static_rates = [r['logical_error_rate'] for r in static_error_rates]

t_stat, p_value = stats.ttest_ind(rt_rates, static_rates)

print(f"\nStatistical Comparison:")
print(f"  RT mean error rate: {np.mean(rt_rates):.4f} ± {np.std(rt_rates):.4f}")
print(f"  Static mean error rate: {np.mean(static_rates):.4f} ± {np.std(static_rates):.4f}")
print(f"  Improvement: {(np.mean(static_rates) - np.mean(rt_rates)) / np.mean(static_rates) * 100:.1f}%")
print(f"  t-statistic: {t_stat:.3f}")
print(f"  p-value: {p_value:.4f}")
print(f"  Significant (p<0.05): {'Yes' if p_value < 0.05 else 'No'}")

## 2.8 Visualize Results

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Logical error rate over time (RT vs Static)
ax1 = axes[0, 0]
iterations = range(N_ITERATIONS)
ax1.plot(iterations, rt_rates, 'o-', label='Real-Time', color='blue', markersize=8)
ax1.plot(iterations, static_rates, 's--', label='Static', color='red', markersize=8)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Logical Error Rate')
ax1.set_title('RT vs Static Selection Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: T1 drift for selected qubits
ax2 = axes[0, 1]
for qubit in candidate_qubits[:5]:  # Plot top 5 qubits
    t1_series = []
    for entry in drift_tracking_data:
        qd = next((q for q in entry['probe_data'] if q['qubit'] == qubit), None)
        if qd and qd.get('t1_probe'):
            t1_series.append(qd['t1_probe'])
    if t1_series:
        ax2.plot(range(len(t1_series)), t1_series, 'o-', label=f'Q{qubit}')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('T1 (µs)')
ax2.set_title('T1 Drift Over Experiment')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Error rate distribution comparison
ax3 = axes[1, 0]
ax3.boxplot([rt_rates, static_rates], labels=['RT', 'Static'])
ax3.set_ylabel('Logical Error Rate')
ax3.set_title('Error Rate Distribution')
ax3.grid(True, alpha=0.3)

# Plot 4: Drift magnitude histogram
ax4 = axes[1, 1]
drift_pcts = df_drift['t1_drift_pct'].dropna()
ax4.hist(drift_pcts, bins=10, edgecolor='black', alpha=0.7)
ax4.axvline(drift_pcts.mean(), color='red', linestyle='--', label=f'Mean: {drift_pcts.mean():.1f}%')
ax4.set_xlabel('T1 Drift (%)')
ax4.set_ylabel('Count')
ax4.set_title('T1 Drift Magnitude Distribution')
ax4.legend()

plt.tight_layout()
plt.savefig('../data/figures/phase2_drift_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 2.9 Save Phase 2 Results

In [ ]:
from src.utils import save_experiment_results

phase2_results = {
    'drift_tracking_data': drift_tracking_data,
    'rt_qec_results': rt_qec_results,
    'static_qec_results': static_qec_results,
    'drift_analysis': df_drift.to_dict('records'),
    'comparison_stats': {
        'rt_mean': float(np.mean(rt_rates)),
        'rt_std': float(np.std(rt_rates)),
        'static_mean': float(np.mean(static_rates)),
        'static_std': float(np.std(static_rates)),
        'improvement_pct': float((np.mean(static_rates) - np.mean(rt_rates)) / np.mean(static_rates) * 100),
        't_statistic': float(t_stat),
        'p_value': float(p_value)
    },
    'experiment_metadata': {
        'phase': 2,
        'n_iterations': N_ITERATIONS,
        'wait_between': WAIT_BETWEEN_ITERATIONS,
        'code_distance': CODE_DISTANCE,
        'backend': backend.name,
        'timestamp': datetime.now(timezone.utc).isoformat()
    }
}

save_experiment_results(phase2_results, '../data/experiments/phase2_drift_results.json')
df_comparison.to_parquet('../data/experiments/phase2_comparison.parquet', index=False)

print("Phase 2 results saved successfully!")

## 2.10 Phase 2 Summary

### Key Findings
- **Drift magnitude**: Quantified intra-day drift in T1, T2, and readout errors
- **RT improvement**: Real-time probe refresh shows X% improvement over static selection
- **Probe overhead**: 30-shot probes add minimal QPU time (~Y seconds per iteration)

### Next Steps (Phase 3)
- Implement drift-aware qubit selection that predicts future drift
- Use probe history to weight qubit selection
- Integrate drift information into the decoder for adaptive-prior decoding